In [ ]:
# Imports
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from snmfem.estimators import SmoothNMF
from snmfem.datasets import built_in_datasets as bi

## Not done yet

In [ ]:
bi.generate_built_in_datasets()

In [ ]:
particules = bi.load_particules(sample = 0)
gb = bi.load_grain_boundary(sample=0)

# Dataset creation (optional)

/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
## **Les modifs principales sur la création des données sont dans generate_data.ipynb et je suppose que ca devrait rester comme ca.**
## Mon idée etant que : Soit on veut les données par défaut et on load Dataset_A, etc... (pas encore fait du coup), Soit on veut créer des données spécifiques auquel cas on utilise ``generate_data.ipynb``
/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

# Data loading and problem definition

In [ ]:

spim = hs.load("/mnt/c/Users/teurtrie/Travail/SNMF_EDXS/generated_datasets/built_in_particules/sample_1.hspy")
# spim.set_signal_type("EDXSsnmfem")

In [ ]:
spim.metadata

# Problem solving

Full hyperspy

In [ ]:
# problem_type: "identity", "no_brstlg", "bremsstralung"
G,shape_2d = spim.extract_params("bremsstrahlung")
phases, weights = spim.extract_truth()

In [ ]:
est = SmoothNMF( n_components = 3,tol=0.0001, max_iter = 100, G = G, shape_2d = shape_2d, lambda_L= 2, true_D = phases, true_A = weights)
out = spim.decomposition(algorithm = est, return_info=True)

In [ ]:
losses = est.get_losses()
# This works but I think we are better off with : P = est.P_
Ps = spim.get_P()
Pe = est.P_

In [ ]:
spim.plot_decomposition_factors(3)

In [ ]:
spim.plot_decomposition_loadings(3)

# Problem Solving

With the usual scikit use

In [ ]:
G,shape_2d = spim.extract_params("bremsstrahlung")
phases, weights = spim.extract_truth()
X = spim.get_Xflat()
est = SmoothNMF(n_components = 3,tol=0.1, G = G, shape_2d = shape_2d, lambda_L= 2, true_D = phases, true_A = weights, hspy_comp = False)
D = est.fit_transform(X)


In [ ]:
losses = est.get_losses()
P = est.P_